In [ ]:

import os
os.environ['NUMEXPR_MAX_THREADS'] = '40'
os.environ['NUMEXPR_NUM_THREADS'] = '40'
import logging
logging.basicConfig()
logging.getLogger('glets').setLevel(logging.INFO)
logging.debug("test")

In [ ]:

import glets
import logging

logging.getLogger().setLevel(logging.INFO)
import importlib
import numpy as np

import matplotlib.pyplot as plt
#get_ipython().run_cell_magic('bash', '', "cat /proc/cpuinfo | grep 'model name' | uniq\ncat /proc/cpuinfo | grep 'cpu family' | uniq\n lscpu | grep 'Model:' | uniq\n lscpu | egrep '^Thread|^Core|^Socket|^CPU\\('\n")

In [ ]:
importlib.reload(glets)
gwfr = glets.Rayfront(wavelength=2.2e-6,size=5e-4,
                samplescheme='fibbonacci')
osys = glets.GaubletOpticalSystem(epd=5e-4,dimd=4e-4,npix=512)
osys.add_aperture(shape='lyot',diameter=5e-4)
osys.add_optic(efl=5e-3)
osys.add_distance(distance=5e-3,index=1)
#osys.add_aperture(shape='fpm',diameter=2.5e-4)
osys.add_detector()
%time gwfr.propogate(elements=osys.elements)

In [ ]:

# Set up figure
plt.figure(figsize=[7,7])
plt.imshow(np.log(np.abs(gwfr.Ephase)))
plt.colorbar()
plt.show()

# Lyot stop needs to be shifted on-axis
# it's the same index responsible for both lists (y >= 0, x >= 0)

In [ ]:
%time gwfr.propogate(elements=osys.elements)

In [ ]:
%prun gwfr.propogate(elements=osys.elements)

In [ ]:
import numexpr as ne
import numpy as np
import numba 
np.__file__,ne.__file__, numba.__file__

In [ ]:
import numpy as np
z=np.ones([512,512,2000])+np.pi*1j


In [ ]:
import numba
import cmath
dtype=numba.complex128

@numba.vectorize([dtype(dtype)],
                 target="parallel")
def numba_exp(x):
        return cmath.exp(x)
    

@numba.vectorize([dtype(dtype)],
                 target="cuda")
def gpu_numba_exp(x):
        return cmath.exp(x)

In [ ]:
%time ez = poppy.accel_math._exp(z)
%time eznp = np.exp(z)
%time ez = ne.evaluate("exp(z)")

In [ ]:
%time nez=numba_exp(z)

In [ ]:
try:
    get_ipython().run_cell_magic('bash', '', 'nvidia-smi\n')

except Exception as err:
    print(err)
